In [1]:
#imports
import cv2
import numpy as np
import time
from collections import deque

In [2]:
#Values to be set
datasetPath='C:/Users/Lucrezia/Desktop/Internship/MNM_version2-Revision17/Mingle/'# put the path of your video here 
DAY='1'
cameranum='1'
N=2 #number of persons to be track
lenhistory=124 # len of the tracker history, if it is shown in the frame

In [3]:
def suc(succ,box,pts,frame,pl=False):
    if succ:
        (x1,y1,w1,h1) = [int(numero) for numero in box]
        cv2.rectangle(frame,(x1,y1), (x1+w1,y1+h1),(0,255,0),2)

        xc = int(x1 + w1 / 2)
        yc = int(y1 + h1 / 2)
        cntr = (xc, yc)

        pts.appendleft(cntr)
        frame=cv2.circle(frame, (xc,yc), radius=2, color=(255, 255, 255), thickness=3)
        
        if pl== True:
            for l in range(1, len(pts)):
                if pts[l - 1] is None or pts[l] is None:
                    continue
                frame=cv2.line(frame,pts[l - 1],pts[l],(0, 255, 0),2)
    return (frame,pts)

In [4]:
def creat(frame,N):
    my_pts = {}
    my_track ={}
    my_b_box={}
    for i in range(0,N):
        my_trackname="track%d" % i
        my_track[my_trackname] = cv2.TrackerMIL_create()
        my_ptsname="pts%d" % i
        my_pts[my_ptsname] = deque(maxlen=lenhistory) 
        my_b_boxname="b_box%d" % i
        my_b_box[my_b_boxname] =cv2.selectROI('Frame',frame)
    return(my_track,my_pts,my_b_box)

In [5]:
def create2(frame,N,my_track):
    my_succ={}
    my_box={}
    for i in range(0,N):
        mytrackanme="track%d" % i
        mysucname="succ%d" % i
        myboxname="box%d" % i
        my_succ[mysucname],my_box[myboxname] = my_track[mytrackanme].update(frame)
    return(my_succ,my_box)

In [6]:
def create3(frame,N,my_track,b_box):
    for i in range(0,N):
        mytrackanme="track%d" % i
        b_boxname="b_box%d" % i
        my_track[mytrackanme].init(frame,b_box[b_boxname])

In [8]:
# Loading video 
cap = cv2.VideoCapture(datasetPath+'videos/downsample/30min_day'+DAY+'_cam'+cameranum+'_20fps_960x540.mp4')
frame_w=int(cap.get(3))
frame_h=int(cap.get(4))
outvideo = cv2.VideoWriter('output8.mp4',cv2.VideoWriter_fourcc('M','P','4','V'), 25, (frame_w,frame_h))
ret, frame = cap.read()
cv2.imshow('Frame', frame)  

tracks,pts,b_boxs=creat(frame,N)
create3(frame,N,tracks,b_boxs)

font = cv2.FONT_HERSHEY_SIMPLEX
numero=0
#Let's start
seconds = time.time()

while cap.isOpened() and numero<1000:
    numero=numero+1
    ret, frame = cap.read()
    if not ret:
        break
    succ,box=create2(frame,N,tracks)
    height, width, channels = frame.shape

    for i in range(0,N):
        my_ptsname="pts%d" % i
        mysucname="succ%d" % i
        myboxname="box%d" % i
        frame,pts[my_ptsname]=suc(succ[mysucname],box[myboxname],pts[my_ptsname],frame,pl=False)#If you put True here, you will have a history of the tracker shown in the frame 
    cv2.imshow("Image", frame)
    outvideo.write(frame)
    cv2.waitKey(1)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
outvideo.release()
cv2.destroyAllWindows()
seconds2 = time.time()
print('total time: ',seconds2-seconds)

total time:  9.886993646621704
